# Deploying Machine Learning Model using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.

In [32]:
!kubectl label nodes apiserver-crd role=locust --overwrite

node "apiserver-crd" not labeled


## Start seldon-core

In [33]:
!helm install ../../helm-charts/seldon-core --name seldon-core --set grafana_prom_admin_password=password --set persistence.enabled=false --set cluster_manager.image.tag=0.2.16_kubeapi --set apife.image.tag=0.0.7_kubeapi

NAME:   seldon-core
LAST DEPLOYED: Tue Oct 31 18:43:11 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                           TYPE    DATA  AGE
cluster-manager-client-secret  Opaque  1     8s
grafana-prom-secret            Opaque  1     8s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           8s

==> v1/Pod
NAME         READY  STATUS             RESTARTS  AGE
zookeeper-1  0/1    ContainerCreating  0         7s
zookeeper-2  0/1    Pending            0         6s
zookeeper-3  0/1    Pending            0         6s

==> v1/Pod(related)
grafana-prom-import-dashboards-nw6c8      0/1  ContainerCreating  0  8s
alertmanager-deployment-779f586d4d-jcfxt  0/1  ContainerCreating  0  8s
seldon-apiserver-55c4558dcd-7tl6r         0/1  ContainerCreating  0  8s
seldon-cluster-manager-85846c959d-qf2nx   0/1  ContainerCreating  0  8s
grafana-prom-deployment-586bc9b9c5-gbmmm  0/1  ContainerCreating  0  7s
kafka-

In [34]:
!kubectl get deployments

NAME                      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
alertmanager-deployment   1         1         1            1           50s
grafana-prom-deployment   1         1         1            1           50s
kafka                     1         1         1            1           49s
prometheus-deployment     1         1         1            1           49s
redis                     1         1         1            1           49s
seldon-apiserver          1         1         1            1           50s
seldon-cluster-manager    1         1         1            1           50s


# Integrating with Kubernetes API

In [35]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

## Create ML Deployment

In [36]:
!kubectl apply -f resources/model.json

mldeployment "mldeployment-example" created


In [37]:
!kubectl get mldeployments

NAME                   AGE
mldeployment-example   2s


In [38]:
!kubectl describe mldeployments mldeployment-example 

Name:         mldeployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"MLDeployment","metadata":{"annotations":{},"labels":{"app":"seldon"},"name":"mldeployment-ex...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         MLDeployment
Metadata:
  Cluster Name:                   
  Creation Timestamp:             2017-10-31T18:44:15Z
  Deletion Grace Period Seconds:  <nil>
  Deletion Timestamp:             <nil>
  Initializers:                   <nil>
  Resource Version:               7420
  Self Link:                      /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/mldeployments/mldeployment-example
  UID:                            7ee3ff6c-be6b-11e7-97cc-08002751d93c
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Cluster:
  Id:              1
  Name:            test deployment
  Oauth _ Key:     oauth-key
  Oaut

In [41]:
!kubectl get rs --selector=app=sd-1-p

NAME               DESIRED   CURRENT   READY     AGE
sd-1-p-c78c8f77f   1         1         1         23s


## Get predictions

In [42]:
%%bash
API_SERVER=35.195.94.148:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
#SERVER=0.0.0.0:8080
echo "$SERVER"
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

192.168.99.100:30032
{
  "meta": {
    "puid": "7ofokc7o2k7cn4maat14ocau6u",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Update deployment with canary

In [43]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

In [44]:
!kubectl apply -f resources/model_with_canary.json

mldeployment "mldeployment-example" configured


In [59]:
!kubectl get rs --selector=app=sd-1-p

NAME                DESIRED   CURRENT   READY     AGE
sd-1-c-54f5db79cd   1         1         1         49s
sd-1-p-c78c8f77f    1         1         1         1m


In [60]:
%%bash
API_SERVER=35.189.249.200:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

{
  "meta": {
    "puid": "q7hkhb00ojgrqsopgui0i80hf",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Load test

In [61]:
!helm install --set oauth.key=oauth-key --set oauth.secret=oauth-secret --name loadtest ../../helm-charts/seldon-core-loadtesting

NAME:   loadtest
LAST DEPLOYED: Tue Oct 31 18:46:08 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP  EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.0.0.49   <none>       5557:31707/TCP,5558:31154/TCP,8089:30199/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-sqdml   0/1    ContainerCreating  0         0s
locust-master-1-wzb5d  0/1    ContainerCreating  0         0s




# Tear down

In [62]:
!kubectl delete -f resources/model_with_canary.json

mldeployment "mldeployment-example" deleted


In [63]:
!helm delete loadtest --purge

release "loadtest" deleted


In [64]:
!helm delete seldon-core --purge

release "seldon-core" deleted
